In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
!pip install pytorch_lightning

     |████████████████████████████████| 582 kB 4.3 MB/s 
     |████████████████████████████████| 398 kB 76.0 MB/s 
     |████████████████████████████████| 136 kB 91.1 MB/s 
     |████████████████████████████████| 596 kB 81.4 MB/s 
     |████████████████████████████████| 1.1 MB 71.4 MB/s 
     |████████████████████████████████| 271 kB 75.3 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |████████████████████████████████| 144 kB 77.9 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow

In [ ]:
!pip install transformers
!pip install dataset

     |████████████████████████████████| 3.8 MB 4.2 MB/s 
     |████████████████████████████████| 895 kB 62.5 MB/s 
     |████████████████████████████████| 6.5 MB 61.6 MB/s 
     |████████████████████████████████| 67 kB 6.8 MB/s 
     |████████████████████████████████| 210 kB 4.3 MB/s 
     |████████████████████████████████| 78 kB 8.7 MB/s 


In [ ]:
import pickle
with open('./drive/MyDrive/Data/English_TGD2_TGD_All_data.pkl','rb') as file:
    TGD_All_data=pickle.load(file)


#Topic prediction model

In [ ]:
import os
import math

import torch
from torch.nn import BCELoss,BCEWithLogitsLoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, ElectraTokenizer, ElectraModel, ElectraConfig

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
device =torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') 


In [ ]:
pre_tained_model=ElectraModel.from_pretrained('google/electra-small-discriminator')

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/51.7M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#model_save_path_topic="drive/MyDrive/Models/TopicPrediction/TGD_Classification_Concatenate_Both_Topic_Entent_Sigmoid_Less_Data.bin" # lat utterance
model_save_path_topic="drive/MyDrive/Models/TopicPrediction/TGD_Classification_Concatenate_Both_Topic_Entent_Sigmoid.bin" # lat utterance
tokenizer_topic_path="drive/MyDrive/Models/TopicPrediction/Tokenizer/" # lat utterance


In [ ]:
tokenizer_topic=ElectraTokenizer.from_pretrained(tokenizer_topic_path)

In [ ]:

pre_tained_model.resize_token_embeddings(len(tokenizer_topic)) 

Embedding(31238, 128)

In [ ]:
import pickle 
with open('drive/MyDrive/Models/TopicPrediction/topic_list.pkl','rb') as f :
  topic_list=pickle.load(f)

In [ ]:
class ElectraForMultiLabelSequenceClassification(torch.nn.Module):
  
  def __init__(self, pre_trained_model,num_labels_topics=2,num_labels_entent=2):
    super(ElectraForMultiLabelSequenceClassification, self).__init__()
    self.num_labels_topics = num_labels_topics
    self.electra = pre_trained_model#Use the pretrqined model
    self.layer1 = torch.nn.Linear(256, 1000)# add a linear layer on top of the encoding  discriminator layers of Electra 
    self.layer2  = torch.nn.Linear(1000, 2000)# add a linear layer on top of the encoding  discriminator layers of Electra 
    self.layer3  = torch.nn.Linear(2000, 3000)#change it to 5000 # add a linear layer on top of the encoding  discriminator layers of Electra 
    self.classifier_topics = torch.nn.Linear(3000, self.num_labels_topics)# add a linear layer on top of the encoding  discriminator layers of Electra 
    self.drop=torch.nn.Dropout(p=0.4)
    self.leaky=torch.nn.ReLU()

    torch.nn.init.xavier_normal_(self.classifier_topics.weight) #initialise the weight of th linear layer
    torch.nn.init.xavier_normal_(self.layer1.weight) #initialise the weight of th linear layer
    torch.nn.init.xavier_normal_(self.layer2.weight) #initialise the weight of th linear layer
    torch.nn.init.xavier_normal_(self.layer3.weight) #initialise the weight of th linear layer

  def forward(self, input_ids, token_type_ids=None,\
              attention_mask=None, labels_topic=None,labels_entent=None):
    # last hidden layer
    last_hidden_state = self.electra(input_ids=input_ids,\
                                   attention_mask=attention_mask,\
                                   token_type_ids=token_type_ids)
    # pool the outputs into a mean vector
    mean_last_hidden_state = self.pool_hidden_state(last_hidden_state)
    l1 = self.leaky(self.layer1(mean_last_hidden_state))
    l1=self.drop(l1)
    l2 = self.leaky(self.layer2(l1))
    l3 = self.leaky(self.layer3(l2))
    l3=self.drop(l3)
    logits_topics = torch.sigmoid(self.classifier_topics(l3))
        
    if labels_topic is not None:
      loss_fct_topic = BCELoss()
      loss1 = loss_fct_topic(logits_topics.view(-1, self.num_labels_topics),\
                      labels_topic.view(-1, self.num_labels_topics))
      
      
      return loss1
    else:
      return logits_topics


  def pool_hidden_state(self, last_hidden_state):
    """
    Pool the output vectors into a single mean vector 
    """
    last_hidden_state = last_hidden_state[0]
    mean_last_hidden_state = torch.mean(last_hidden_state, 1)
    return mean_last_hidden_state
    
  

# len(Y_train[0]) = 6


In [ ]:
#This code is borrowed from this link : https://towardsdatascience.com/multi-label-text-classification-with-xlnet-b5f5755302df

checkpoint = torch.load(model_save_path_topic)
model_state_dict_topic = checkpoint['state_dict']
model_topic = ElectraForMultiLabelSequenceClassification(pre_tained_model,num_labels_topics=model_state_dict_topic["classifier_topics.weight"].size()[0])
model_topic.load_state_dict(model_state_dict_topic)

<All keys matched successfully>

In [ ]:
#This code is borrowed from this link : https://towardsdatascience.com/multi-label-text-classification-with-xlnet-b5f5755302df

def generate_predictions_topic(model, features,mask, num_labels, device="cpu", batch_size=32):
  num_iter = math.ceil(features.shape[0]/batch_size)
  
  pred_probs = np.array([]).reshape(0, num_labels)
  
  model.to(device)
  model.eval()
  
  for i in range(num_iter):
    X = features[i*batch_size:(i+1)*batch_size,:]
    masks = mask[i*batch_size:(i+1)*batch_size,:]
    X = X.to(device)
    masks = masks.to(device)
    with torch.no_grad():
      logits = model(input_ids=X, attention_mask=masks)
      #logits = logits.sigmoid().detach().cpu().numpy()
      logits = logits.detach().cpu().numpy()
      pred_probs = np.vstack([pred_probs, logits])
  
  return pred_probs
  


In [ ]:
def tokenize_inputs_topic(liste, tokenizer, num_embeddings=512):
    
    outputs = tokenizer.batch_encode_plus(liste,truncation=True,return_tensors='pt',return_attention_mask=True,max_length=512, padding='max_length')
    # convert tokenized text into numeric ids for the appropriate LM
    input_ids =outputs.input_ids
    

    return (input_ids,outputs.attention_mask)

In [ ]:
def get_all_prediction(list_text,topic_list):
  preds_vector=[]
  preds_word=[]
  for text in list_text:
    test_input_ids,test_attention_masks=tokenize_inputs_topic([text], tokenizer_topic, num_embeddings=512)
    pred_probs = generate_predictions_topic(model_topic, test_input_ids,test_attention_masks, 2464, device=device, batch_size=32)
    prob=np.round(pred_probs)
    prob=list(prob[0])
    preds_vector.append(prob)

    pred_probs=prob
    indexes=[i for i, v in enumerate(prob) if v==1]
    predicted_topics=" ".join(topic_list[x] for x in indexes)
    predicted_topics=" <topic> "+predicted_topics+" </topic>"
    preds_word.append(predicted_topics)
  
  return preds_vector,predicted_topics
  
#preds,preds_word=get_all_prediction(test_data,topic_list)

In [ ]:
t="<seeker> how about you recommend me a moving movie first ?"
preds,preds_word=get_all_prediction([t],topic_list)
preds_word

' <topic> moving talk </topic>'

#Profile generation model

In [ ]:
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5TokenizerFast as T5Tokenizer)

In [ ]:
path_tokenizer="./drive/MyDrive/Summarization/PresonaChat-ProfileGeneration_whith_t5/tokenizer"
path_model="./drive/MyDrive/Summarization/PresonaChat-ProfileGeneration_whith_t5/model"

In [ ]:
tokenizer_t5=T5Tokenizer.from_pretrained(path_tokenizer)
trained_model_t5=T5ForConditionalGeneration.from_pretrained(path_model)

In [ ]:
text="hello what are you doing today ? i am good , i just got off work and tired , i have two jobs . i just got done watching a horror movie i rather read , i've read about 20 books this year . wow ! i do love a good horror movie . loving this cooler weather but a good movie is always good . yes ! my son is in junior high and i just started letting him watch them too i work in the movies as well . neat ! ! i used to work in the human services field yes it is neat , i stunt double , it is so much fun and hard work . yes i bet you can get hurt . my wife works and i stay at home nice , i only have one parent so now i help out my mom . i bet she appreciates that very much . she raised me right , i am just like her . my dad was always busy working at home depot"

In [ ]:
def summarize(text,trained_model,tokenizer_t5):
  text_encoding=tokenizer_t5(text,padding="max_length",max_length=512,truncation=True,return_attention_mask=True,return_tensors="pt")
  generated_ids=trained_model.generate(
      input_ids=text_encoding["input_ids"],
      attention_mask=text_encoding["attention_mask"],
      max_length=150, 
      num_beams=15,
      early_stopping=True,
      repetition_penalty=2.5,
      length_penalty=1.0,
  )
  preds=[tokenizer_t5.decode(gen_id,skip_special_tokens=True,clean_up_tokenization_space=True) for gen_id in generated_ids]
  return " ".join(preds)

# Personality prediction model

In [ ]:
model_name='google/electra-small-discriminator'

tokenizer_personality = ElectraTokenizer.from_pretrained(model_name, do_lower_case=True)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

In [ ]:
from torch.nn import BCEWithLogitsLoss,MSELoss


In [ ]:
class ElectraSharedWeights_Multi_Task(torch.nn.Module):
  
  def __init__(self, r_num_labels=2,c_num_labels=2):
    super(ElectraSharedWeights_Multi_Task, self).__init__()
    self.c_num_labels = c_num_labels#The number of the classification labels whiche are 4 labels to use them as the final size for  the final linear layer
    self.r_num_labels = r_num_labels#The number of the regression labels whiche are 5 labelsto use them as the final size for  the final linear layer
    self.electra = ElectraModel.from_pretrained(model_name)#Instanciate  the pretrained model
    self.layer1 = torch.nn.Linear(256, 150)# add The first linear layer on top of the encoding  discriminator layers of Electra 
    self.layer2  = torch.nn.Linear(150, 100)# add the second linear layer on top of the first one
    self.layer3  = torch.nn.Linear(100, 50)# add the third linear layer on top of the second one
    #self.normilizer1  = torch.nn.BatchNorm1d(256)# add the normalizer to the electra values
    #self.normilizer2  = torch.nn.BatchNorm1d(100)# add the  normilizer layer onto the second layer values
    #All the previous layers will be shared by the last separate two layers (classifier layer and regressor layer)

    self.classifier = torch.nn.Linear(50, c_num_labels)# add the final linear layer for classification on top of the third layer   
    self.regressor = torch.nn.Linear(50, r_num_labels)# add the final linear layer for regression on top of the third layer 
    self.drop=torch.nn.Dropout(p=0.4)# Define a dropout layer to overpass the overfitting during the training phase
    self.leaky=torch.nn.LeakyReLU()# Use the Leaky Relu activation function as the ;ain activation function for the shared layers



    #In order to train our models the weights need to be initialized, because if they were at 0, then the model will not be optimized so we need to
    # inilize them. The most popular way to inisialize the weights is to use the Xavier initializer
    torch.nn.init.xavier_normal_(self.layer1.weight) #initialise the weight of the first linear layer
    torch.nn.init.xavier_normal_(self.layer2.weight) #initialise the weight of the second linear layer
    torch.nn.init.xavier_normal_(self.layer3.weight) #initialise the weight of the third linear layer
    torch.nn.init.xavier_normal_(self.classifier.weight) #initialise the weight of the classification layer
    torch.nn.init.xavier_normal_(self.regressor.weight) #initialise the weight of th regression layer

  def forward(self, input_ids, token_type_ids=None,\
              attention_mask=None, labels_r=None,labels_c=None):
    #Now that we defined the architecture and initialized the weights we need to define the forward function
    #and how each defined variable will be used in the architecture

    # last hidden layer
    #for ech row we will use the ELECTRa model to create an embedding vector given the tokenized input
    Electra_hidding_state = self.electra(input_ids=input_ids,\
                                   attention_mask=attention_mask,\
                                   token_type_ids=token_type_ids)
    contextual_embedding = Electra_hidding_state[0]
    contextual_embedding = torch.mean(contextual_embedding, 1)
    #contextual_embedding=self.normilizer1(contextual_embedding)
    # pool the outputs into a mean vector
    cl1 = self.leaky(self.layer1(contextual_embedding))
    cl1=self.drop(cl1)
    cl2 = self.leaky(self.layer2(cl1))
    #cl2 = self.normilizer2(cl2)
    cl3 = self.leaky(self.layer3(cl2))
    cl3=self.drop(cl3)

    #All the previous weights will be shared by both last layers (classifier and regresor) 
    clogits = self.classifier(cl3)# The classification results(MBTI Labels) given the shared weights

    rlogits = self.regressor(cl3)# The regression results(BIG5 labels) given the shared weights


    # Due to the fact that we have a multi-task learning model then we will have two different losses
    # one to compute the classification loss and the other is to compute the regression loss
    # We will use the BCE version for the classification loss and the MSE for the regression loss
        
    closs_fct =BCEWithLogitsLoss()# Classification loss
    rloss_fct = MSELoss()# Regression loss

    if ((labels_r is not None) and (labels_c is not None)):

  
      closs = closs_fct(clogits.view(-1, self.c_num_labels),\
                        labels_c.view(-1, self.c_num_labels))
      rloss=rloss_fct(rlogits.view(-1, self.r_num_labels),\
                        labels_r.view(-1, self.r_num_labels))
      #Because a model need to have one loss function to be trained on and be cause we need to minimise both regression and classification loss
      #we create a combination of the regression loss and the classification loss by summing them and deviding by 2
      #By combining both losses  into a one loss we will force the model to minimize the combination loss wich will lead to the minimization of the 
      #regression and the classification loss (minimizing their sum will lead to minimizing their original values)
      loss=(rloss+closs)/2
      return loss
    else:
      return clogits,rlogits



In [ ]:
model_save_path="./drive/MyDrive/Models/Pandora_Personality/SeedDoubleHead_SharedWeights_20epochs_Electra.bin"

In [ ]:
#This code is borrowed from this link : https://towardsdatascience.com/multi-label-text-classification-with-xlnet-b5f5755302df

checkpoint = torch.load(model_save_path)
model_state_dict = checkpoint['state_dict']
model_personality = ElectraSharedWeights_Multi_Task(5,4)
model_personality.load_state_dict(model_state_dict)

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [ ]:
def Input_Tokenization_personality(liste, tokenizer):
    
    outputs = tokenizer.batch_encode_plus(liste,padding=True,truncation=True,return_tensors='pt',return_attention_mask=True)    

    return (outputs.input_ids,outputs.attention_mask)

In [ ]:
#This code is borrowed from this link : https://towardsdatascience.com/multi-label-text-classification-with-xlnet-b5f5755302df

def Predict(model,tokenizer, text, num_label_c,num_label_r, device="cpu", batch_size=32):
  features,mask=Input_Tokenization_personality([text], tokenizer)
  num_iter = math.ceil(features.shape[0]/batch_size)# define the number of iteration in all the data by batches
  features.to(device)
  mask.to(device)
  model.eval() #Indicate that we will use the model to evaluation purposes

  c_pred_probs = np.array([]).reshape(0, num_label_c)# Create dummpy classification predictions 'all 0' vectors that we will modify
  r_pred_probs = np.array([]).reshape(0, num_label_r)# Create dummpy regression predictions 'all 0' vectors that we will modify
  model.to(device)
  d={}
  
  for i in range(num_iter):# for each batch
    X = features[i*batch_size:(i+1)*batch_size,:]#Get the features of the passed data
    masks = mask[i*batch_size:(i+1)*batch_size,:]#Get the masks of the passed data
    X = X.to(device)#transfer them to the device 'exmp cuda'
    masks = masks.to(device)#transfer them to the device 'exmp cuda'
    with torch.no_grad():
      logits_c,logits_r = model(input_ids=X, attention_mask=masks) # predict the values of the passed batch of data
      #logits = logits.sigmoid().detach().cpu().numpy()
      logits_c = logits_c.sigmoid().detach().cpu().numpy()# for the classification head we will aplly a sigmoid function as like we said the classification labels are binary labels
      c_pred_probs = np.vstack([c_pred_probs, logits_c])# stack them to the pr-defined dummy classification variable
      logits_r = logits_r.detach().cpu().numpy()# For the regression values wi well use directly the linear layer output because the regression labels are real values
      r_pred_probs = np.vstack([r_pred_probs, logits_r])# stack them to the pr-defined dummy regression variable
  d['class_pred']=np.round(c_pred_probs) # to get binary values for the sigmoids probability values we will use the .round() function for the classification results
  d['reg_pred']=r_pred_probs
  
  return d
  


#Next movie to watch

In [ ]:
from transformers import ElectraTokenizerFast, ElectraForMaskedLM,ElectraConfig, ElectraTokenizer

from transformers import DistilBertTokenizer, DistilBertForMaskedLM


In [ ]:
class Electra4RecWithTGD_OnlyNextItemToPredict(torch.nn.Module):
  
  def __init__(self):
    super(Electra4RecWithTGD_OnlyNextItemToPredict, self).__init__()
    
    model=DistilBertForMaskedLM.from_pretrained('distilbert-base-uncased')
    model.resize_token_embeddings(len(tokenizer_4rec))


    model_mask_last=DistilBertForMaskedLM.from_pretrained('distilbert-base-uncased')
    model_mask_last.resize_token_embeddings(len(tokenizer_4rec))


    self.electra = model
    self.electra_mask_last = model_mask_last



   
   
  def forward(self, input_ids, token_type_ids=None,attention_mask=None, labels=None,input_ids_mask_last=None, token_type_ids_mask_last=None,attention_mask_mask_last=None, labels_mask_last=None):
    #Now that we defined the architecture and initialized the weights we need to define the forward function
    #and how each defined variable will be used in the architecture
    
    
    
    position_ids=torch.arange(start=0, end = input_ids.shape[1], step=1).expand_as(input_ids).to(device)
    
    
    #position_sinusoidal_embeddings = np.array([[pos / np.power(10000, 2 * (j // 2) / input_ids.shape[1]) for j in range(input_ids.shape[1])] for pos in range(input_ids.shape[0])])
    #position_ids=position_sinusoidal_embeddings
   
   
    # last hidden layer
    #for ech row we will use the ELECTRa model to create an embedding vector given the tokenized input


    #Electra predfoned positions

    MLM_outputs = self.electra(input_ids, attention_mask=attention_mask,labels=labels)
    self.electra_mask_last.state_dict()['vocab_layer_norm.bias']=self.electra.state_dict()['vocab_layer_norm.bias']

    MLM_outputs_mask_last= self.electra_mask_last(input_ids_mask_last, attention_mask=attention_mask_mask_last,labels=labels_mask_last)

    
    #withour position  embdedding
    #MLM_outputs = self.electra(input_ids, attention_mask=attention_mask,labels=labels)

    

    if ((labels is not None) ):


      loss=(MLM_outputs.loss + MLM_outputs_mask_last.loss)
      return loss
    else:
      return MLM_outputs.logits,MLM_outputs_mask_last.logits



In [ ]:
path='drive/MyDrive/Models/NextMovie/DistilBert_TGD_Ensemble_no_BIG5_last_masked_and_NormalMask_OnlyNextItemToPredict.bin'
path_tokenizer='drive/MyDrive/Models/NextMovie/Tokenizer/'


In [ ]:
tokenizer_4rec = DistilBertTokenizer.from_pretrained(path_tokenizer)

In [ ]:
#This code is borrowed from this link : https://towardsdatascience.com/multi-label-text-classification-with-xlnet-b5f5755302df

checkpoint = torch.load(path)
model_state_dict = checkpoint['state_dict']
model_movie = Electra4RecWithTGD_OnlyNextItemToPredict()
model_movie.load_state_dict(model_state_dict)

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

<All keys matched successfully>

In [ ]:
def sequence_Input_Tokenization_movie_prediction(liste, tokenizer):
    
    outputs = tokenizer(liste,truncation=True,return_tensors='pt',return_attention_mask=True,max_length=512, padding='max_length')    
    outputs['labels']=outputs.input_ids.detach().clone()
    return outputs

In [ ]:
#This code is borrowed from this link : https://towardsdatascience.com/multi-label-text-classification-with-xlnet-b5f5755302df

def Predict_Movie(sequences,model, tokenizer_m, device="cpu", batch_size=32):
  num_iter = math.ceil(np.array(sequences).shape[0]/batch_size)# define the number of iteration in all the data by batches
  model.eval() #Indicate that we will use the model to evaluation purposes
  model.to(device)
  d={}
  
  IndexToKey={ value:key for key,value in tokenizer_m.get_vocab().items() }
  for i in range(num_iter):
    inputs_i=tokenizer_m(sequences[i],return_tensors='pt')
    inputs_ids=inputs_i.input_ids.to(device)
    inputs_mask=inputs_i.attention_mask.to(device)

    with torch.no_grad():
        
        #output= model(inputs_ids,inputs_mask,input_ids_mask_last=inputs_ids,attention_mask_mask_last=inputs_mask)# predict the values of the passed batch of data
        output= model(input_ids=inputs_ids,attention_mask=inputs_mask,input_ids_mask_last=inputs_ids,attention_mask_mask_last=inputs_mask)# predict the values of the passed batch of data
        #logits = logits.sigmoid().detach().cpu().numpy()
        out=torch.argmax(output[0],dim=-1).to(device)
        
        ch=''
        
        for ind in list(out.detach().tolist()[0]):
          ch=ch+IndexToKey[ind]+' '
  d['MLM_movie_prediction']=ch.split()
  
  return d['MLM_movie_prediction'][-2]
  


# Dialog Generation model

In [ ]:
from transformers import (
    MODEL_WITH_LM_HEAD_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)


In [ ]:
def remove_movie_from_path_goal(TGD_All_data,typee='train'):
  for i, dial in enumerate(TGD_All_data[typee]):
    goal_path=dial['goal_path']
    for k, v in goal_path.items():
      v=[x for x in v if x[0]!='@']
      goal_path[k]=v
    
    TGD_All_data[typee][i]['goal_path']=goal_path
  return TGD_All_data[typee]

TGD_All_data['train']=remove_movie_from_path_goal(TGD_All_data,typee='train')
TGD_All_data['test']=remove_movie_from_path_goal(TGD_All_data,typee='test')
TGD_All_data['valid']=remove_movie_from_path_goal(TGD_All_data,typee='valid')

In [ ]:
def create_entent_topic(TGD_All_data=TGD_All_data,typee="train"):

  topic_list=TGD_All_data["real_topics"].values()
  for i,dial in enumerate(TGD_All_data[typee]):
    goal_path=dial["goal_path"]
    new_goal_path={}
    
    for k,v in goal_path.items():
      topics=[x for x in v if x in topic_list]
      intents=[x for x in v if x not in topic_list]
      if(len(topics)==0):
        topics=["no-topic"]
      new_goal_path[k]={"intents":intents,"topics":intents+topics}
    TGD_All_data[typee][i]["goal_path"]=new_goal_path

  return TGD_All_data
TGD_All_data=create_entent_topic(TGD_All_data=TGD_All_data,typee="train")
TGD_All_data=create_entent_topic(TGD_All_data=TGD_All_data,typee="test")
TGD_All_data=create_entent_topic(TGD_All_data=TGD_All_data,typee="valid")





In [ ]:
import re
import random
def create_path_included_data(data,typee='valid'):
  data_dialogue_rows=[]
  mentioned_movies=[]
  movie_history_watching=[]
  for i,dial in enumerate(data[typee]):
    goal_path=dial['goal_path']
    messages=dial['messages']
    mentionMovies=dial["mentionMovies"]
    movies_interaction=dial["user_history_movies_interaction_ids"]
    movie_history_watching.append(movies_interaction)


    visited_profiles=[]
    for j in range(1):
      profile_list=dial["user_profile"]
      personality_list=dial["BIG5"]
      shuffled_profile=random.sample(profile_list,len(profile_list))
      while shuffled_profile in visited_profiles:
        shuffled_profile=random.sample(profile_list,len(profile_list))
      personality= " ".join(str(int(p)) for p in personality_list )
      
      profile=" .".join(p for p in shuffled_profile )
      lines=[]
      lines.append('<big5> '+personality+" </big5>")

      
      lines.append('<profile> '+profile+" </profile>")
      movies_to_add=[]
      '''if len(movies_interaction)==0:
        movies_to_add=[" "]*3

      if len(movies_interaction)==1:
        movies_to_add=[movies_interaction[-1]]*3

      if len(movies_interaction)==2:
        movies_to_add=list(movies_interaction)+[movies_interaction[-1]]

      if len(movies_interaction)>2:
        movies_to_add=list(movies_interaction[-3:])'''

      for i,v in enumerate(messages):
        index=i+1
        if index in mentionMovies:
          curenntmovi=mentionMovies[index]["id"]
          #print(curenntmovi)
          movies_to_add.append(curenntmovi)
          #print(movies_to_add)
          last_three=movies_to_add[-3:]
          random.shuffle(last_three)
          concatenated_movies= " ".join(x for x in last_three)
          #lines[-1]=lines[-1]+" <movies> "+curenntmovi+" </movies>"
          m='<'+v['role'].lower()+'>'+' '+v['content']


        else:
          m='<'+v['role'].lower()+'>'+' '+v['content']
        m=m+' <topic> '+' '.join(x for x in goal_path[v['local_id']]["topics"] )+" </topic> "
        m=re.sub(' +', ' ', m).strip()
        if("@" in m ):
          lin= np.array([word for word in m.split(" ") ])
          indexes=[i for i,v in enumerate(lin) if v[0]=="@"]
          elements=[x for x in list(lin[indexes]) if x is not None]
          mentioned_movies=mentioned_movies+elements

        lines.append(m)
    
    
    
    

    
      data_dialogue_rows.append(lines)
    
  return data_dialogue_rows,mentioned_movies,movie_history_watching
    

train_rows,train_mentioned_movies,train_movie_history_watching=create_path_included_data(TGD_All_data,typee='train')   
test_rows,test_mentioned_movies,test_movie_history_watching=create_path_included_data(TGD_All_data,typee='test')   
valid_rows,valid_mentioned_movies,valid_movie_history_watching=create_path_included_data(TGD_All_data,typee='valid')

In [ ]:
path_model='drive/MyDrive/Models/DialogGeneration/Prompt_based_TGD_small_keep_all_data2.bin'
path_tokenizer='drive/MyDrive/Models/DialogGeneration/Tokenizer/'


In [ ]:
tokenizer_gpt = AutoTokenizer.from_pretrained(path_tokenizer)


In [ ]:
class DialogGPT(torch.nn.Module):
  
  def __init__(self,tokenizer):
    super(DialogGPT, self).__init__()
    
    self.dialogGPT = AutoModelForCausalLM.from_pretrained('microsoft/DialoGPT-small')
    self.dialogGPT.resize_token_embeddings(len(tokenizer))

   

   
  def forward(self, input_ids, token_type_ids=None,\
              attention_mask=None, labels=None): 
   
        
    outputs = self.dialogGPT(input_ids,attention_mask=attention_mask, labels=labels)
    loss = outputs[0]
      
      
    return loss


  def pool_hidden_state(self, last_hidden_state):
    """
    Pool the output vectors into a single mean vector 
    """
    last_hidden_state = last_hidden_state[0]
    mean_last_hidden_state = torch.mean(last_hidden_state, 1)
    return mean_last_hidden_state
    
  

# len(Y_train[0]) = 6

In [ ]:
#model_save_path_gpt="drive/MyDrive/Models/DialogGeneration/Prompt_based_TGD_small_keep_all_data.bin"


In [ ]:
#This code is borrowed from this link : https://towardsdatascience.com/multi-label-text-classification-with-xlnet-b5f5755302df

checkpoint_gpt = torch.load(path_model)
model_state_dict_gpt = checkpoint_gpt['state_dict']
model_gpt = DialogGPT(tokenizer=tokenizer_gpt)
model_gpt.load_state_dict(model_state_dict_gpt)
model_gpt.to(device)

Downloading:   0%|          | 0.00/641 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/335M [00:00<?, ?B/s]

DialogGPT(
  (dialogGPT): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(86054, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0): GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (1): GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
        

# last utterance topic prediction

In [ ]:
movies_dict= TGD_All_data['all_movies']


In [ ]:
def tokenize_inputs(liste, tokenizer, num_embeddings=1024):
    
    outputs = tokenizer.batch_encode_plus(liste,truncation=True,return_tensors='pt',return_attention_mask=True,max_length=num_embeddings, padding='max_length')
    # convert tokenized text into numeric ids for the appropriate LM
    input_ids =outputs.input_ids
    outputs["labels"]=input_ids
    

    return input_ids,outputs.attention_mask
    

In [ ]:
i=5
persona_0=test_rows[i][0]+tokenizer_gpt.eos_token+test_rows[i][1]
conv_0=test_rows[i][2:]
movie_history_watching=test_movie_history_watching[i]

# changing the topic

In [ ]:
i=5
persona_0=test_rows[i][0]+tokenizer_gpt.eos_token+test_rows[i][1]
conv_0=test_rows[i][2:]
movie_history_watching=test_movie_history_watching[i]

In [ ]:
persona_0

'<big5> 28 79 21 50 60 </big5><|endoftext|><profile> i like sports .the place i want to travel most is india .i like animation .i like kids .i like love very much .i like to live on the ground .i like writing love letters very much .i like music very much .i think i am a loud voice .i really want to have sincere feelings </profile>'

In [ ]:
[movies_dict[x]['name']for x in movie_history_watching[-10:]]
#lemon mouth : children movie 
#the teacher and the stray cat : comedy family
#dumb boy: family movie

['song of sunshine',
 'air evacuation',
 'the teacher and the stray cat',
 'legend no. 17',
 'spiritual grace',
 'the mind of the han and han princess',
 'lemon mouth',
 'undead debt collector',
 'live with integrity',
 'dumb boy']

In [ ]:

#Beam search Decoding
next_movie_to_watch=""
previous_string=persona_0+tokenizer_gpt.eos_token +'<recommender>'
persona_ids=tokenizer_gpt.encode(previous_string , return_tensors='pt').to(device)
new_user_input_ids=persona_ids
previous_interactions=""
#print(f'previous string: {previous_string}')
# Let's chat for 5 lines
for step in range(100):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    # print(new_user_input_ids)
    #chat_history_ids= tokenizer.encode(persona)
    # append the new user input tokens to the chat history
    bot_input_ids = new_user_input_ids

    #print(f"bot_input_ids : {tokenizer_gpt.decode(bot_input_ids[0])}")

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model_gpt.dialogGPT.generate(
        bot_input_ids, max_length=1000,
        pad_token_id=tokenizer_gpt.eos_token_id,
        no_repeat_ngram_size=5,
        num_beams=5,
        early_stop=True

        
    )
    #print(f'chat history id : {tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0])}')

#        do_sample=True,

#num_return_sequences=3
    
    # pretty print last ouput tokens from bot
    response=tokenizer_gpt.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    #print(f'response oroginale ; {response}')


    if "<topic>" in response:
        in_response=response
        response=response.split()
        response=response[:response.index("<topic>")]
        response=" ".join(x for x in response)
    

      
    if '@' in response:
      movie_pred=re.findall(r"@[0-9]+",response)[0]
      previous_interactions=previous_interactions+' '+movie_pred


    #print(f"see all the dialogue {all_dialogue_text}")
    #print(f"Fed input to the topic prediction {response}")
    _,topics=get_all_prediction([response],topic_list)
    #_,topics=get_all_prediction_only_topic([topic_text_without_big5_personality],topic_list,intent_tokens)
    #print(f" predict the topic {topics}")
    
    if '@' in response:
      movie_pred=re.findall(r"@[0-9]+",response)[0]
      #response=response+f" you may also like {movies_dict[next_movie_to_watch]['name']} based on your previous history of watching."
      print(">>DialoGPT:      {}".format(re.sub(movie_pred, "\x1b[6;30;42m"+movies_dict[movie_pred]['name']+"\x1b[0m", response)))
    else:
      print(">>DialoGPT:      {}".format(response))

    
    

    
    user_utterance='<seeker> '+input(">> User:         ")
    _,topics_user_utterance=get_all_prediction([user_utterance],topic_list)
    #print(f"\n User topic {topics_user_utterance}\n")
    user_utterance=user_utterance

    """if ("request" in topics) and ("recommendation " in topics):
      previous_interactions=interactions
      next_movie_to_watch=Predict_Movie([previous_interactions+' [MASK]'],big5=[Big5_personality],model=model_movie,tokenizer_m=tokenizer_4rec,device=device, batch_size=32)
      print(next_movie_to_watch)
      previous_interactions=previous_interactions+' '+next_movie_to_watch"""
    previous_string=previous_string+' '+in_response+tokenizer_gpt.eos_token+ user_utterance + topics_user_utterance+tokenizer_gpt.eos_token+ '<recommender>'
    new_user_input_ids = tokenizer_gpt.encode(previous_string ,return_tensors='pt').to(device)
    
    
    
    
    if (('end' in topics_user_utterance) and ("feedback" in topics_user_utterance) and ("no-topic" in topics_user_utterance)):
      break



interac=" ".join(x for x in movie_history_watching)+previous_interactions+"[MASK]"
per=[int(x) for x in users_personalities[0].split()]
final_shot=Predict_Movie([interac],model=model_movie,tokenizer_m=tokenizer_4rec,device=device, batch_size=32)
print(f"DialoGPT: you may also like {movies_dict[final_shot]['name']} based on your previous watching history. Have a good one, be safe, bye.")



>>DialoGPT:      how are things?
>>DialoGPT:      no problem, i recommend you to watch red light, which is a story about love and death.
>>DialoGPT:      of course, i recommend a comedy movie for you.
>>DialoGPT:      then i recommend a horror comedy movie to you. have you ever watched horror wax museum? it's a bit scary, but it's still very funny.
>>DialoGPT:      no, i don’t have one, but i have watched a lot of horror movies. would you like to recommend a horror movie to you?
>>DialoGPT:      of course, i would recommend a horror and thriller movie to you. you will like horror wax museum. the ending is a bit unexpected, but the plot is still very good.
>>DialoGPT:      then i recommend a movie about horror and horror. have you seen horror wax museum? the plot is very exciting, and the actors' acting skills are also very good. it is also a horror movie. it is worth watching. it is a rare horror movie that is not bloody and not bloody. it is worth recommending.


In [ ]:

#Beam search Decoding
next_movie_to_watch=""
previous_string=persona_0+tokenizer_gpt.eos_token +'<recommender>'
persona_ids=tokenizer_gpt.encode(previous_string , return_tensors='pt').to(device)
new_user_input_ids=persona_ids
previous_interactions=""
#print(f'previous string: {previous_string}')
# Let's chat for 5 lines
for step in range(100):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    # print(new_user_input_ids)
    #chat_history_ids= tokenizer.encode(persona)
    # append the new user input tokens to the chat history
    bot_input_ids = new_user_input_ids

    #print(f"bot_input_ids : {tokenizer_gpt.decode(bot_input_ids[0])}")

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model_gpt.dialogGPT.generate(
        bot_input_ids, max_length=1000,
        pad_token_id=tokenizer_gpt.eos_token_id,
        no_repeat_ngram_size=5,
        num_beams=5,
        early_stop=True

        
    )
    #print(f'chat history id : {tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0])}')

#        do_sample=True,

#num_return_sequences=3
    
    # pretty print last ouput tokens from bot
    response=tokenizer_gpt.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    #print(f'response oroginale ; {response}')


    if "<topic>" in response:
        in_response=response
        response=response.split()
        response=response[:response.index("<topic>")]
        response=" ".join(x for x in response)
    

      
    if '@' in response:
      movie_pred=re.findall(r"@[0-9]+",response)[0]
      previous_interactions=previous_interactions+' '+movie_pred


    #print(f"see all the dialogue {all_dialogue_text}")
    #print(f"Fed input to the topic prediction {response}")
    _,topics=get_all_prediction([response],topic_list)
    #_,topics=get_all_prediction_only_topic([topic_text_without_big5_personality],topic_list,intent_tokens)
    #print(f" predict the topic {topics}")
    
    if '@' in response:
      movie_pred=re.findall(r"@[0-9]+",response)[0]
      #response=response+f" you may also like {movies_dict[next_movie_to_watch]['name']} based on your previous history of watching."
      print(">>DialoGPT:      {}".format(re.sub(movie_pred, "\x1b[6;30;42m"+movies_dict[movie_pred]['name']+"\x1b[0m", response)))
    else:
      print(">>DialoGPT:      {}".format(response))

    
    

    
    user_utterance='<seeker> '+input(">> User:         ")
    _,topics_user_utterance=get_all_prediction([user_utterance],topic_list)
    #print(f"\n User topic {topics_user_utterance}\n")
    user_utterance=user_utterance

    """if ("request" in topics) and ("recommendation " in topics):
      previous_interactions=interactions
      next_movie_to_watch=Predict_Movie([previous_interactions+' [MASK]'],big5=[Big5_personality],model=model_movie,tokenizer_m=tokenizer_4rec,device=device, batch_size=32)
      print(next_movie_to_watch)
      previous_interactions=previous_interactions+' '+next_movie_to_watch"""
    previous_string=previous_string+' '+in_response+tokenizer_gpt.eos_token+ user_utterance + topics_user_utterance+tokenizer_gpt.eos_token+ '<recommender>'
    new_user_input_ids = tokenizer_gpt.encode(previous_string ,return_tensors='pt').to(device)
    
    
    
    
    if (('end' in topics_user_utterance) and ("feedback" in topics_user_utterance) and ("no-topic" in topics_user_utterance)):
      break



interac=" ".join(x for x in movie_history_watching)+previous_interactions+"[MASK]"
per=[int(x) for x in users_personalities[0].split()]
final_shot=Predict_Movie([interac],model=model_movie,tokenizer_m=tokenizer_4rec,device=device, batch_size=32)
print(f"DialoGPT: you may also like {movies_dict[final_shot]['name']} based on your previous watching history. Have a good one, be safe, bye.")



# with another personality


In [ ]:
i=5
persona_0=test_rows[i][0]+tokenizer_gpt.eos_token+test_rows[i][1]
conv_0=test_rows[i][2:]
movie_history_watching=test_movie_history_watching[i]
persona_0

'<big5> 28 79 21 50 60 </big5><|endoftext|><profile> i like to live on the ground .the place i want to travel most is india .i like kids .i like writing love letters very much .i like animation .i like sports .i like music very much .i really want to have sincere feelings .i think i am a loud voice .i like love very much </profile>'

In [ ]:
i=5
persona_0=test_rows[45][0]+tokenizer_gpt.eos_token+test_rows[i][1]
conv_0=test_rows[i][2:]
movie_history_watching=test_movie_history_watching[i]
persona_0

'<big5> 53 95 48 61 52 </big5><|endoftext|><profile> i like to live on the ground .the place i want to travel most is india .i like kids .i like writing love letters very much .i like animation .i like sports .i like music very much .i really want to have sincere feelings .i think i am a loud voice .i like love very much </profile>'

In [ ]:

#Beam search Decoding
next_movie_to_watch=""
previous_string=persona_0+tokenizer_gpt.eos_token +'<recommender>'
persona_ids=tokenizer_gpt.encode(previous_string , return_tensors='pt').to(device)
new_user_input_ids=persona_ids
previous_interactions=""
#print(f'previous string: {previous_string}')
# Let's chat for 5 lines
for step in range(100):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    # print(new_user_input_ids)
    #chat_history_ids= tokenizer.encode(persona)
    # append the new user input tokens to the chat history
    bot_input_ids = new_user_input_ids

    #print(f"bot_input_ids : {tokenizer_gpt.decode(bot_input_ids[0])}")

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model_gpt.dialogGPT.generate(
        bot_input_ids, max_length=1000,
        pad_token_id=tokenizer_gpt.eos_token_id,
        no_repeat_ngram_size=5,
        num_beams=5,
        early_stop=True

        
    )
    #print(f'chat history id : {tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0])}')

#        do_sample=True,

#num_return_sequences=3
    
    # pretty print last ouput tokens from bot
    response=tokenizer_gpt.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    #print(f'response oroginale ; {response}')


    if "<topic>" in response:
        in_response=response
        response=response.split()
        response=response[:response.index("<topic>")]
        response=" ".join(x for x in response)
    

      
    if '@' in response:
      movie_pred=re.findall(r"@[0-9]+",response)[0]
      previous_interactions=previous_interactions+' '+movie_pred


    #print(f"see all the dialogue {all_dialogue_text}")
    #print(f"Fed input to the topic prediction {response}")
    _,topics=get_all_prediction([response],topic_list)
    #_,topics=get_all_prediction_only_topic([topic_text_without_big5_personality],topic_list,intent_tokens)
    #print(f" predict the topic {topics}")
    
    if '@' in response:
      movie_pred=re.findall(r"@[0-9]+",response)[0]
      #response=response+f" you may also like {movies_dict[next_movie_to_watch]['name']} based on your previous history of watching."
      print(">>DialoGPT:      {}".format(re.sub(movie_pred, "\x1b[6;30;42m"+movies_dict[movie_pred]['name']+"\x1b[0m", response)))
    else:
      print(">>DialoGPT:      {}".format(response))

    
    

    
    user_utterance='<seeker> '+input(">> User:         ")
    _,topics_user_utterance=get_all_prediction([user_utterance],topic_list)
    #print(f"\n User topic {topics_user_utterance}\n")
    user_utterance=user_utterance

    """if ("request" in topics) and ("recommendation " in topics):
      previous_interactions=interactions
      next_movie_to_watch=Predict_Movie([previous_interactions+' [MASK]'],big5=[Big5_personality],model=model_movie,tokenizer_m=tokenizer_4rec,device=device, batch_size=32)
      print(next_movie_to_watch)
      previous_interactions=previous_interactions+' '+next_movie_to_watch"""
    previous_string=previous_string+' '+in_response+tokenizer_gpt.eos_token+ user_utterance + topics_user_utterance+tokenizer_gpt.eos_token+ '<recommender>'
    new_user_input_ids = tokenizer_gpt.encode(previous_string ,return_tensors='pt').to(device)
    
    
    
    
    if (('end' in topics_user_utterance) and ("feedback" in topics_user_utterance) and ("no-topic" in topics_user_utterance)):
      break



interac=" ".join(x for x in movie_history_watching)+previous_interactions+"[MASK]"
per=[int(x) for x in users_personalities[2].split()]
final_shot=Predict_Movie([interac],model=model_movie,tokenizer_m=tokenizer_4rec,device=device, batch_size=32)
print(f"DialoGPT: you may also like {movies_dict[final_shot]['name']} based on your previous watching history. Have a good one, be safe, bye.")


#this user has a litle bit lower neurosisme factor EST, which makes him more relaxed person

>>DialoGPT:      what are you doing?
>> User:         i'm looking for a thriller movie, can you recommend me one, please
>>DialoGPT:      then i highly recommend you to watch cold blood horror, which is a good movie, but i don't like it.
>> User:         you know what i think i want to watch a comedy movie to laugh a little bit. do you have any recommendations
>>DialoGPT:      then i recommend you to watch the mummy. this is a horror movie, but it is not a horror movie. it is a funny and funny movie. it is worth watching.
>> User:         i love this movie, its a comedy and an adventure movie type, my best I will check it later . however, i would like to watch an inspirational movie for now
>>DialoGPT:      then i recommend a thrilling and funny movie to you. have you ever watched peachy furious car? this is a suspenseful and suspenseful comedy with a good plot and a good rhythm.
>> User:         i think i changed my mind, is there any movie about suspense!
>>DialoGPT:      then i high

"\n>>DialoGPT:      what are you doing?\n>> User:         i'm looking for a thriller movie, can you recommend me one, please.\n>>DialoGPT:      i highly recommend the movie cold blood horror, which is a bit like a horror movie.\n>> User:         you know what i think i want to watch a comedy movie to laugh a little bit. do you have any recommendations.\n>>DialoGPT:      i recommend you to watch the mummy. this is a horror movie, but it is not a comedy. it is a good movie. it is worth watching.\n>> User:          i love this movie, its a comedy and an adventure movie type, my best I will check it later . however, i would like to watch an inspirational movie for now\n>>DialoGPT:      you can watch morning anchor, a relaxed and inspirational film, which is very suitable for you. it is a relaxing and inspirational film.\n>> User:          i think i changed my mind, is there any movie about suspense!\n>>DialoGPT:      i highly recommend picnic on the cliff, a suspenseful and suspenseful sus

# Full dialog Scenario

In [ ]:

#Beam search Decoding
next_movie_to_watch=""
previous_string=persona_0+tokenizer_gpt.eos_token +'<recommender>'
persona_ids=tokenizer_gpt.encode(previous_string , return_tensors='pt').to(device)
new_user_input_ids=persona_ids
previous_interactions=""
#print(f'previous string: {previous_string}')
# Let's chat for 5 lines
for step in range(100):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    # print(new_user_input_ids)
    #chat_history_ids= tokenizer.encode(persona)
    # append the new user input tokens to the chat history
    bot_input_ids = new_user_input_ids

    #print(f"bot_input_ids : {tokenizer_gpt.decode(bot_input_ids[0])}")

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model_gpt.dialogGPT.generate(
        bot_input_ids, max_length=1000,
        pad_token_id=tokenizer_gpt.eos_token_id,
        no_repeat_ngram_size=5,
        num_beams=5,
        early_stop=True

        
    )
    #print(f'chat history id : {tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0])}')

#        do_sample=True,

#num_return_sequences=3
    
    # pretty print last ouput tokens from bot
    response=tokenizer_gpt.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    #print(f'response oroginale ; {response}')


    if "<topic>" in response:
        in_response=response
        response=response.split()
        response=response[:response.index("<topic>")]
        response=" ".join(x for x in response)
    

      
    if '@' in response:
      movie_pred=re.findall(r"@[0-9]+",response)[0]
      previous_interactions=previous_interactions+' '+movie_pred


    #print(f"see all the dialogue {all_dialogue_text}")
    #print(f"Fed input to the topic prediction {response}")
    _,topics=get_all_prediction([response],topic_list)
    #_,topics=get_all_prediction_only_topic([topic_text_without_big5_personality],topic_list,intent_tokens)
    #print(f" predict the topic {topics}")
    
    if '@' in response:
      movie_pred=re.findall(r"@[0-9]+",response)[0]
      #response=response+f" you may also like {movies_dict[next_movie_to_watch]['name']} based on your previous history of watching."
      print(">>DialoGPT:      {}".format(re.sub(movie_pred, "\x1b[6;30;42m"+movies_dict[movie_pred]['name']+"\x1b[0m", response)))
    else:
      print(">>DialoGPT:      {}".format(response))

    
    

    
    user_utterance='<seeker> '+input(">> User:         ")
    _,topics_user_utterance=get_all_prediction([user_utterance],topic_list)
    #print(f"\n User topic {topics_user_utterance}\n")
    user_utterance=user_utterance

    """if ("request" in topics) and ("recommendation " in topics):
      previous_interactions=interactions
      next_movie_to_watch=Predict_Movie([previous_interactions+' [MASK]'],big5=[Big5_personality],model=model_movie,tokenizer_m=tokenizer_4rec,device=device, batch_size=32)
      print(next_movie_to_watch)
      previous_interactions=previous_interactions+' '+next_movie_to_watch"""
    previous_string=previous_string+' '+in_response+tokenizer_gpt.eos_token+ user_utterance + topics_user_utterance+tokenizer_gpt.eos_token+ '<recommender>'
    new_user_input_ids = tokenizer_gpt.encode(previous_string ,return_tensors='pt').to(device)
    
    
    
    
    if (('end' in topics_user_utterance) and ("feedback" in topics_user_utterance) and ("no-topic" in topics_user_utterance)):
      break



interac=" ".join(x for x in movie_history_watching)+previous_interactions+"[MASK]"
per=[int(x) for x in users_personalities[0].split()]
final_shot=Predict_Movie([interac],big5=[per],model=model_movie,tokenizer_m=tokenizer_4rec,device=device, batch_size=32)
print(f"DialoGPT: you may also like {movies_dict[final_shot]['name']} based on your previous watching history. Have a good one, be safe, bye.")


NameError: ignored